In [50]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import *
import pyvista as pv
from matplotlib.colors import ListedColormap
import time

In [51]:
earthquake_data = pv.read('cleaned_earthquake_data.vtk')
earthquake_data

PolyData (0x28582563e20)
  N Cells:    1
  N Points:   31500
  N Strips:   0
  X Bounds:   -1.272e+07, -1.211e+07
  Y Bounds:   4.404e+06, 5.236e+06
  Z Bounds:   0.000e+00, 6.628e+00
  N Arrays:   31

In [58]:
active_years = []
for year in earthquake_data.point_data['Year']:
    if year not in active_years:
        active_years.append(year)
active_years.sort()
active_years

[1850,
 1853,
 1859,
 1868,
 1872,
 1873,
 1874,
 1876,
 1877,
 1878,
 1880,
 1881,
 1883,
 1884,
 1885,
 1887,
 1889,
 1891,
 1893,
 1894,
 1895,
 1896,
 1899,
 1900,
 1901,
 1902,
 1903,
 1905,
 1906,
 1908,
 1909,
 1910,
 1913,
 1914,
 1915,
 1916,
 1918,
 1919,
 1920,
 1921,
 1923,
 1924,
 1925,
 1926,
 1927,
 1928,
 1929,
 1930,
 1931,
 1932,
 1933,
 1934,
 1935,
 1936,
 1937,
 1938,
 1939,
 1940,
 1941,
 1942,
 1943,
 1944,
 1945,
 1946,
 1947,
 1948,
 1949,
 1950,
 1951,
 1952,
 1953,
 1954,
 1955,
 1956,
 1957,
 1958,
 1959,
 1960,
 1961,
 1962,
 1963,
 1964,
 1965,
 1966,
 1967,
 1968,
 1969,
 1970,
 1971,
 1972,
 1973,
 1974,
 1975,
 1976,
 1977,
 1978,
 1979,
 1980,
 1981,
 1982,
 1983,
 1984,
 1985,
 1986,
 1987,
 1988,
 1989,
 1990,
 1991,
 1992,
 1993,
 1994,
 1995,
 1996,
 1997,
 1998,
 1999,
 2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016]

In [53]:
for mag in earthquake_data.point_data['Mag']:
    if mag < 0:
        print(mag)

In [3]:
def get_coords_by_year(year):
    lat_lng = []
    indices = np.where(earthquake_data.point_data['Year'] == year)[0]
    for i in indices:
        x, y, z = float(earthquake_data.point_data['Long'][i]), earthquake_data.point_data['Lat'][i], 0
        lat_lng.append((x, y, z))
    lat_lng_np = np.array(lat_lng)        
    return lat_lng_np

def get_mags_by_year(year):
    mags = []
    indices = np.where(earthquake_data.point_data['Year'] == year)[0]
    for i in indices:
        mags.append(earthquake_data.point_data['Mag'][i])
    mags_np = np.array(mags)
    return mags_np.flatten()

utah_corners = np_points = np.array([[-114.04172399259161, 41.99372594774304, 0.0], 
                      [-111.0466887774167, 42.0015686697003, 0.0],
                      [-111.04673083487624, 40.997968805895084, 0.0],
                      [-109.05007687780909, 41.000660502664616, 0.0],
                      [-109.04522189716712, 36.999084296202696, 0.0],
                      [-114.05060037628137, 37.00039575681357, 0.0]])

In [37]:
c1 = np.array([0.99, 0.0, 0.0])
c2 = np.array([0.0, 0.99, 0.0])
c3 = np.array([0.0, 0.0, 0.99])
c4 = np.array([0.99, 0.99, 0.0])
c5 = np.array([0.99, 0.0, 0.99])
c6 = np.array([0.0, 0.99, 0.99])

def get_cmap():
    mapping = np.linspace(earthquake_data.point_data['Mag'].min(), earthquake_data.point_data['Mag'].max(), 256)
    mapping
    new_colors = np.empty((256, 3))
    new_colors[mapping >= 8.0] = c1
    new_colors[mapping < 7.9] = c2
    new_colors[mapping < 6.9] = c3
    new_colors[mapping < 6.0] = c4
    new_colors[mapping < 6.0] = c5
    new_colors[mapping < 2.5] = c6

    mag_classification_cmap = ListedColormap(new_colors)
    mag_classification_cmap

    lut = pv.LookupTable(cmap=mag_classification_cmap)
    return lut

In [42]:
# Set up legend based on classifications
legend_entries = []
legend_entries.append(['<Usually not felt', c6])
legend_entries.append(['Minor damage', c5])
legend_entries.append(['Slight damage', c4])
legend_entries.append(['Lots of damage', c3])
legend_entries.append(['Serious damage', c2])
legend_entries.append(['Destroys entire communities', c1])

In [43]:
def create_mag_mesh(year, pl):
    mag_mesh = pv.UnstructuredGrid()
    mag_mesh.points = get_coords_by_year(year)
    mag_mesh.point_data["Mag"] = get_mags_by_year(year)
    mag_cmap = get_cmap()
    pl.add_mesh(mag_mesh, point_size=1)
    ball = pv.Sphere(radius=0.05, theta_resolution=35, phi_resolution=35)
    mag_glyphs = mag_mesh.glyph(geom=ball, orient=False, scale="Mag")
    pl.add_mesh(mag_glyphs, cmap=mag_cmap)
    return

In [49]:
pl = pv.Plotter(notebook=False)
    
# Display approximate boundaries for Utah
outline_mesh = pv.PolyData(utah_corners)
pl.add_mesh(outline_mesh, point_size=15)
poly_line = pv.MultipleLines(points=utah_corners)
pl.add_mesh(poly_line, line_width=2)
line = pv.Line(pointa=utah_corners[0], pointb=utah_corners[5])
pl.add_mesh(line, line_width=2, color='white')
pl.add_legend(legend_entries)
create_mag_mesh(1983, pl)
pl.show(cpos='xy')